In [40]:
from pyspark.sql import SparkSession
from delta import *

import subprocess
import json
import os

In [42]:
from uuid import uuid4

In [43]:
session_id = str(uuid4())

In [ ]:
builder = SparkSession.builder.appName(session_id) \
    .master("spark://spark-master:7077")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()    
spark.sparkContext.setLogLevel("WARN")

In [29]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer, Consumer, KafkaError
from confluent_kafka import avro
import pyspark.sql.functions as pyspark_f
import pyspark.sql.types as pyspark_dt

# Create Topic & Schema Registry

In [ ]:
topic_name = 'sample_topic'
kafka_config = {
    'bootstrap.servers': 'kafka-1:9092,kafka-2:9092,kafka-3:9092',
    'schema.registry.url': 'http://schema-registry:8085' # URL for Schema Registry
}

In [ ]:
# Create Kafka topic
admin_client = AdminClient(kafka_config)
new_topic = NewTopic(topic_name, num_partitions=2, replication_factor=2)

# Create the topic
admin_client.create_topics([new_topic])
print(f"Kafka topic {topic_name} created.")

Kafka topic 'example_topic' created.


In [38]:
# Define the Avro schema
value_schema_str = '''
{
    "namespace": "example.avro",
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}
'''
# Register the Avro schema with the Schema Registry
schema_registry_client = avro.CachedSchemaRegistryClient(kafka_config['schema.registry.url'])
try:
    value_schema = json.loads(value_schema_str)
    schema_registry_client.register(f"{topic_name}-value", value_schema)
    print(f"Schema for topic {topic_name} registered.")
except SchemaParseException as e:
    print(f"Failed to parse schema: {e}")

/tmp/ipykernel_7565/2994693135.py:14: DeprecationWarning: CachedSchemaRegistry constructor is being deprecated. Use CachedSchemaRegistryClient(dict: config) instead. Existing params ca_location, cert_location and key_location will be replaced with their librdkafka equivalents as keys in the conf dict: `ssl.ca.location`, `ssl.certificate.location` and `ssl.key.location` respectively
  schema_registry_client = avro.CachedSchemaRegistryClient(kafka_config['schema.registry.url'])


NameError: name 'SchemaParseException' is not defined